In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e16/sample_submission.csv
/kaggle/input/playground-series-s3e16/train.csv
/kaggle/input/playground-series-s3e16/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
def prepare_data(fpath, train=True):
    def normalize(col):
        mean = col.mean()
        std = col.std()
        col -= mean
        col /= std
        return col
    df = pd.read_csv(fpath)
    df = pd.get_dummies(df, columns=["Sex"])
    df.pop("id")
    if train:
        y = df.pop("Age")
    else:
        y = None
    X = df
    
    X["Volume"] = X["Length"] * X["Height"] * X["Diameter"]
    for col in ["Weight","Shucked Weight","Viscera Weight","Shell Weight","Volume"]:
        X[col] = normalize(X[col])
    return X, y

In [4]:
df = pd.read_csv("/kaggle/input/playground-series-s3e16/train.csv")
df = pd.get_dummies(df, columns=["Sex"])
df.pop("id")
y = df.pop("Age")
X = df

In [5]:
y

0         9
1         8
2         9
3        11
4         8
         ..
74046    10
74047     6
74048    10
74049     8
74050     6
Name: Age, Length: 74051, dtype: int64

In [6]:
def normalize(col):
    mean = col.mean()
    std = col.std()
    col -= mean
    col /= std
    return col

In [7]:
normalize(X["Weight"])

0        0.441801
1       -1.025191
2        0.110075
3        2.156468
4       -0.007598
           ...   
74046    2.156468
74047   -1.022950
74048    0.482147
74049   -0.523119
74050   -1.423039
Name: Weight, Length: 74051, dtype: float64

In [8]:
X

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Sex_F,Sex_I,Sex_M
0,1.5250,1.1750,0.3750,0.441801,12.728926,6.647958,8.348928,0,1,0
1,1.1000,0.8250,0.2750,-1.025191,4.521745,2.324659,3.401940,0,1,0
2,1.3875,1.1125,0.3750,0.110075,11.339800,5.556502,6.662133,0,0,1
3,1.7000,1.4125,0.5000,2.156468,20.354941,10.991839,14.996885,1,0,0
4,1.2500,1.0125,0.3375,-0.007598,11.977664,4.507570,5.953395,0,1,0
...,...,...,...,...,...,...,...,...,...,...
74046,1.6625,1.2625,0.4375,2.156468,20.680960,10.361742,12.332033,1,0,0
74047,1.0750,0.8625,0.2750,-1.022950,4.323299,2.296310,3.543687,0,1,0
74048,1.4875,1.2000,0.4125,0.482147,12.303683,7.540967,8.079607,1,0,0
74049,1.2125,0.9625,0.3125,-0.523119,8.972617,2.919999,4.280774,0,1,0


In [9]:
X["Volume"] = X["Length"] * X["Height"] * X["Diameter"]

In [10]:
for col in ["Weight","Shucked Weight","Viscera Weight","Shell Weight","Volume"]:
    X[col] = normalize(X[col])
X

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Sex_F,Sex_I,Sex_M,Volume
0,1.5250,1.1750,0.3750,0.441801,0.467185,0.569182,0.453373,0,1,0,0.438171
1,1.1000,0.8250,0.2750,-1.025191,-0.993681,-0.978873,-0.926782,0,1,0,-0.966462
2,1.3875,1.1125,0.3750,0.110075,0.219923,0.178362,-0.017224,0,0,1,0.128555
3,1.7000,1.4125,0.5000,2.156468,1.824604,2.124608,2.308079,1,0,0,2.196235
4,1.2500,1.0125,0.3375,-0.007598,0.333461,-0.197232,-0.214954,0,1,0,-0.375911
...,...,...,...,...,...,...,...,...,...,...,...
74046,1.6625,1.2625,0.4375,2.156468,1.882635,1.898987,1.564615,1,0,0,1.257287
74047,1.0750,0.8625,0.2750,-1.022950,-1.029004,-0.989024,-0.887236,0,1,0,-0.948458
74048,1.4875,1.2000,0.4125,0.482147,0.391492,0.888945,0.378236,1,0,0,0.652194
74049,1.2125,0.9625,0.3125,-0.523119,-0.201433,-0.765698,-0.681597,0,1,0,-0.583589


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [12]:
regr = DecisionTreeRegressor(random_state=13)
cross_val_score(regr, X, y, cv=5)

array([0.12379127, 0.12404478, 0.11490272, 0.1227693 , 0.1170813 ])

In [13]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [14]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Normalization(axis=None))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Normalization(axis=None))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Normalization(axis=None))
model.add(tf.keras.layers.Dense(1,activation='linear'))
model.compile(optimizer='adam', loss=['mae'])
# This builds the model for the first time:
history = model.fit(X, y, batch_size=32, epochs=100, validation_split=0.1, callbacks=[callback])

Epoch 1/100
2083/2083 [==============================] - 8s 3ms/step - loss: 1.7361 - val_loss: 1.5224
Epoch 2/100
2083/2083 [==============================] - 6s 3ms/step - loss: 1.5497 - val_loss: 1.4780
Epoch 3/100
2083/2083 [==============================] - 7s 3ms/step - loss: 1.5232 - val_loss: 1.4166
Epoch 4/100
2083/2083 [==============================] - 6s 3ms/step - loss: 1.5096 - val_loss: 1.4341
Epoch 5/100
2083/2083 [==============================] - 6s 3ms/step - loss: 1.4947 - val_loss: 1.4150
Epoch 6/100
2083/2083 [==============================] - 7s 3ms/step - loss: 1.4892 - val_loss: 1.4166
Epoch 7/100
2083/2083 [==============================] - 7s 3ms/step - loss: 1.4829 - val_loss: 1.4187
Epoch 8/100
2083/2083 [==============================] - 6s 3ms/step - loss: 1.4746 - val_loss: 1.4172
Epoch 9/100
2083/2083 [==============================] - 7s 3ms/step - loss: 1.4768 - val_loss: 1.4354
Epoch 10/100
2083/2083 [==============================] - 6s 3ms/step - l

In [15]:
test_X, test_y = prepare_data("/kaggle/input/playground-series-s3e16/test.csv",train=False)
test_X

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Sex_F,Sex_I,Sex_M,Volume
0,1.0500,0.7625,0.2750,-1.174397,-1.153491,-1.197643,-1.125480,0,1,0,-1.072009
1,1.1625,0.8875,0.2750,-0.629460,-0.552328,-0.654328,-0.776627,0,1,0,-0.860583
2,1.2875,0.9875,0.3250,-0.703464,-0.815021,-0.425831,-0.538772,1,0,0,-0.429804
3,1.5500,0.9875,0.3875,0.388652,0.579272,0.528779,0.079650,1,0,0,0.168735
4,1.1125,0.8500,0.2625,-0.925475,-0.820073,-0.933602,-0.955018,0,1,0,-0.978674
...,...,...,...,...,...,...,...,...,...,...,...
49363,1.3000,1.0375,0.3250,-0.565548,-0.612950,0.036241,-0.836090,1,0,0,-0.346177
49364,1.0375,0.7625,0.2625,-1.043209,-1.014567,-1.101166,-0.895554,0,1,0,-1.113627
49365,1.4875,1.1625,0.3625,0.626361,0.225647,0.635411,0.571216,1,0,0,0.280933
49366,1.2375,0.9500,0.2875,-0.617126,-0.719037,-0.481686,-0.499130,1,0,0,-0.680035


In [16]:
test_pred = model.predict(test_X)

1543/1543 [==============================] - 2s 1ms/step


In [17]:
test_orig = pd.read_csv("/kaggle/input/playground-series-s3e16/test.csv")

In [18]:
output = pd.concat([test_orig["id"], pd.DataFrame(test_pred)], axis=1)

In [19]:
output = output.rename(columns={0:"Age"})
output

,id,Age
0,74051,7.158122
1,74052,7.908869
2,74053,10.441444
3,74054,9.329162
4,74055,7.556980
...,...,...
49363,123414,9.020704
49364,123415,7.426793
49365,123416,11.545671
49366,123417,10.053308


In [20]:
output.to_csv("/kaggle/working/crab_submission.csv",index=False)